# 기본 준비

In [269]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import unicodedata
import json
import pandas as pd

In [3]:
webpage = requests.get("https://terms.naver.com/list.nhn?cid=59088&categoryId=59088&page=1")
soup = BeautifulSoup(webpage.content, "html.parser")

## 페이지에서 주소

In [47]:
#주소를 가져오는 메소드
soup.find(attrs={'class':'content_list'}).find('a').get('href')

'/entry.nhn?docId=5662083&cid=59088&categoryId=59096'

In [38]:
# 제목에서만 주소를 가져옴
soup.find(attrs={'class':'content_list'}).find(attrs={'class':'title'})

<strong class="title">
<a href="/entry.nhn?docId=5662083&amp;cid=59088&amp;categoryId=59096" onclick="nclk(this, 'tml.termlist', '', '', 1);">서피스 허브 - 협업에 최적화된 대화면 멀티터치 태블릿</a>
<a class="btn_txt __TLIST_PUT" data-contentsid="5662083" data-servicecode="TERMS" href="#" onclick="nclk(this, 'tml.kadd','', '', 1)">담기</a>
</strong>

In [37]:
len(soup.find(attrs={'class':'content_list'}).find_all(attrs={'class':'title'}))

15

In [50]:
#한 메인 페이지에 포함된 링크 를 구함
for data in soup.find(attrs={'class':'content_list'}).find_all(attrs={'class':'title'}) :
    print(data.find('a').get('href'))

/entry.nhn?docId=5662083&cid=59088&categoryId=59096
/entry.nhn?docId=5661610&cid=59088&categoryId=59096
/entry.nhn?docId=5646659&cid=59088&categoryId=59096
/entry.nhn?docId=5645660&cid=59088&categoryId=59096
/entry.nhn?docId=5644882&cid=59088&categoryId=59096
/entry.nhn?docId=5570670&cid=59088&categoryId=59096
/entry.nhn?docId=5570443&cid=59088&categoryId=59096
/entry.nhn?docId=5570130&cid=59088&categoryId=59096
/entry.nhn?docId=5569373&cid=59088&categoryId=59096
/entry.nhn?docId=5568551&cid=59088&categoryId=59096
/entry.nhn?docId=5551241&cid=59088&categoryId=59096
/entry.nhn?docId=5550804&cid=59088&categoryId=59096
/entry.nhn?docId=5148680&cid=59088&categoryId=59096
/entry.nhn?docId=5145967&cid=59088&categoryId=59096
/entry.nhn?docId=5144656&cid=59088&categoryId=59096


In [121]:
#문서 번호 구하기
target = soup.find(attrs={'class':'content_list'}).find(attrs={'class':'title'}).find('a').get('href')
start = int(target.find("docId="))
end = int(target.find("&cid"))
docId = int(target[start + 6:end])
docId

3570731

## 페이지 주소 구하기

In [51]:
soup.find(attrs={'class':'paginate'})

<div class="paginate" id="paginate">
<span class="prev disabled"><a href="#" onclick="return false;">이전</a><i class="ico_bar"></i></span>
<strong>1</strong>
<a href="/list.nhn?cid=59088&amp;categoryId=59088&amp;page=2" onclick="nclk(this, 'pag.paging', '', '2', 1)">2</a>
<a href="/list.nhn?cid=59088&amp;categoryId=59088&amp;page=3" onclick="nclk(this, 'pag.paging', '', '3', 1)">3</a>
<a href="/list.nhn?cid=59088&amp;categoryId=59088&amp;page=4" onclick="nclk(this, 'pag.paging', '', '4', 1)">4</a>
<a href="/list.nhn?cid=59088&amp;categoryId=59088&amp;page=5" onclick="nclk(this, 'pag.paging', '', '5', 1)">5</a>
<a href="/list.nhn?cid=59088&amp;categoryId=59088&amp;page=6" onclick="nclk(this, 'pag.paging', '', '6', 1)">6</a>
<a href="/list.nhn?cid=59088&amp;categoryId=59088&amp;page=7" onclick="nclk(this, 'pag.paging', '', '7', 1)">7</a>
<a href="/list.nhn?cid=59088&amp;categoryId=59088&amp;page=8" onclick="nclk(this, 'pag.paging', '', '8', 1)">8</a>
<a href="/list.nhn?cid=59088&amp;categ

In [59]:
#현재 페이지 번호 구하기
now = soup.find(attrs={'class':'paginate'}).find('strong').text
now = int(now)
now

1

In [88]:
#현재 페이지가 올바른 페이지인지 확인
int(soup.find(attrs={'class':'paginate'}).find('strong').text) == now

True

## 크롤러

In [227]:
docs = set()

In [1]:
cid = 42346
categoryId = 42346

In [2]:
targetAdress = "https://terms.naver.com/list.nhn?cid=" + str(cid) + "&categoryId=" + str(categoryId) + "&page="
maxRange = 100

In [237]:
for i in range(1, maxRange) :
    webpage = requests.get(targetAdress + str(i))
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    if int(soup.find(attrs={'class':'paginate'}).find('strong').text) != i:
        break;
    
    for data in tqdm(soup.find(attrs={'class':'content_list'}).find_all(attrs={'class':'title'})) :
        target = data.find('a').get('href')
        start = int(target.find("docId="))
        end = int(target.find("&cid"))
        docs.add(int(target[start + 6:end]))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


In [233]:
len(docs)

512

## 한 페이지에서 데이터 가져오기

In [218]:
webpage = requests.get("https://terms.naver.com/entry.nhn?docId=5662083&cid=59088&categoryId=59096")
soup = BeautifulSoup(webpage.content, "html.parser")

In [219]:
#출처 표기 제거
for data in soup.find(attrs={'class':'size_ct_v2'}).find_all(attrs={'class':'se_textView se_mediaCaption'}) :
    print(data.extract())

<div class="se_textView se_mediaCaption">
<span class="se_textarea">| 서피스 허브 &lt;출처: 마이크로소프트&gt;</span>
</div>
<div class="se_textView se_mediaCaption">
<span class="se_textarea">| 서피스 허브 2 &lt;출처: 마이크로소프트&gt;</span>
</div>
<div class="se_textView se_mediaCaption">
<span class="se_textarea">| 협업용으로 만들어진 오피스 기기, 서피스 허브 2 &lt;출처: 마이크로소프트&gt;</span>
</div>
<div class="se_textView se_mediaCaption">
<span class="se_textarea">| 서피스 허브 2는 업무 공간에 따라 가로 혹은 세로로 쓸 수 있다. &lt;출처: 마이크로소프트&gt;</span>
</div>
<div class="se_textView se_mediaCaption">
<span class="se_textarea">| 서피스 허브 2 4대를 가로로 붙인 모습. &lt;출처: 마이크로소프트&gt;</span>
</div>
<div class="se_textView se_mediaCaption">
<span class="se_textarea">| 서피스 허브 2 여러 대를 붙여 하나의 화면으로 쓰거나 별도 화면으로 분리해 쓸 수 있다. &lt;출처: 마이크로소프트&gt;</span>
</div>


In [221]:
#제목
title = unicodedata.normalize("NFKD", soup.find(attrs={'class':'headword'}).get_text(strip=True))
print(title)

서피스 허브


In [224]:
#본문
main = unicodedata.normalize("NFKD", soup.find(attrs={'class':'size_ct_v2'}).get_text(strip=True))
print(main)

요약서피스 허브는 마이크로소프트(MS)가 선보인 협업용 태블릿이다. 협업에 최적화돼 설계됐다. 2016년 1세대 서피스 허브가 2만1999달러에 출시됐다. 2세대인 서피스 허브 2(Surface Hub 2)는  1세대가 출시된 지 2년여 만인 2018년 5월 16일 공개됐다.외국어 표기Surface Hub서피스 허브서피스 허브는 마이크로소프트(MS)가 2015년 1월 22일 최초 공개한 벽걸이형 대형 태블릿이다. 개인용이 아닌 협업용으로 설계됐다. 제품 공식 출시는 최초 공개로부터 15개월 후인 2016년 이뤄졌다. 단순히 크기를 확대한 태블릿이 아니라 협업에 최적화돼 만들어졌다는 점이 특징이다.서비프 허브는 84형 4K 터치 스크린을 가지고 있고 양 측면에는 1080p 카메라가 배치돼 있다.서피스 허브는 회의실이나 컨퍼런스 룸을 겨냥한 기기다. 손가락이나 스타일러스 펜을 이용해 회의 내용을 기록·공유할 수 있다. 일종의 전자칠판인 셈이다. 여기에 음성인식을 지원하고 동작인식 센서가 탑재된 카메라를 달아 오디오·비디오 컨퍼런스를 효과적ᄋ

## 페이지 번호들로 최종 데이터 만들기

In [281]:
datas = list()

In [282]:
for docId in tqdm(docs):
    targetAdress = "https://terms.naver.com/entry.nhn?docId="+ str(docId) + "&cid=" + str(cid) + "&categoryId=" + str(categoryId)
    webpage = requests.get(targetAdress)
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    #출처 표기 제거
    for data in soup.find(attrs={'class':'size_ct_v2'}).find_all(attrs={'class':'se_textView se_mediaCaption'}) :
        data.extract()
        
    #제목
    title = unicodedata.normalize("NFC", soup.find(attrs={'class':'headword'}).get_text(strip=True))
    
    #본문
    main = unicodedata.normalize("NFC", soup.find(attrs={'class':'size_ct_v2'}).get_text(strip=True)) 
    
    dic = {'source' : 'https://terms.naver.com','cid': cid, 'categoryId':categoryId, 'docId': docId,'title': title, 'main':main}
    datas.append(dic)

100%|████████████████████████████████████████████████████████████████████████████████| 512/512 [04:07<00:00,  2.07it/s]


In [283]:
df = pd.DataFrame(datas)

In [284]:
df

,source,cid,categoryId,docId,title,main
0,https://terms.naver.com,59088,59088,3573766,MP4,"인터넷 시대에 발맞춰 태어난 멀티미디어 파일 규격, MP4윈도우(Windows)나도..."
1,https://terms.naver.com,59088,59088,5148680,HEIF(고효율 이미지 파일 형식),요약HEIF는 '고효율 이미지 파일 형식(High Efficiency Image F...
2,https://terms.naver.com,59088,59088,3579922,iOS9,‘iOS9’애플이 2015년 9월 17일부터 모바일 기기용 운영체제(OS) ‘iOS...
3,https://terms.naver.com,59088,59088,3577875,무선 디스플레이 전송 기술,"모바일 기기의 성능은 비약적으로 발전하고 있지만, 여전히 화면이 작다는 것은 치명적..."
4,https://terms.naver.com,59088,59088,3575828,캐리어 어그리게이션,LTE에서는 여러 개의 주파수를 섞어서 더 빠른 속도를 내는 기술이 적용된다.이동통...
...,...,...,...,...,...,...
507,https://terms.naver.com,59088,59088,3571698,DSLR 카메라,필름카메라는 렌즈를 통해 빛을 받아들여 이를 필름에 감광(빛을 쪼여 영상을 새김)시...
508,https://terms.naver.com,59088,59088,3577843,스마트밴드,요즘도 ‘만보계’를 쓰는 이가 있을까. 하루에 1만 걸음 정도는 걸어야 건강하게 살...
509,https://terms.naver.com,59088,59088,3575795,샵메일(#메일),2012년 8월28일 국무회의에서 ‘전자거래기본법 시행령 개정안’이 의결됐다. 일명...
510,https://terms.naver.com,59088,59088,3573750,랜카드,네트워크(인터넷)에 연결되지 않은컴퓨터는 바보상자로 취급될 정도로 현대의 컴퓨터 사...


In [286]:
df.to_excel('naver_tech_it.xlsx')